# Download network list

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

In [ ]:
r = requests.get("http://training.lczero.org/networks/?show_all=1")

In [ ]:
soup = BS(r.content, "html.parser")

In [ ]:
tbl = soup.find("table")

In [ ]:
records = []
columns = []
for tr in tbl.findAll("tr"):
    ths = tr.findAll("th")
    if len(ths) != 0:
        for each in ths:
            columns.append(each.text)
    else:
        trs = tr.findAll("td")
        record = []
        for col in trs:
            try:
                link = col.find("a")["href"]
                record.append(link)
            except:
                text = col.text
                record.append(text)
        records.append(record)
df = pd.DataFrame(data=records, columns=columns)
df = df.astype({"Number": "int32"})
df_all = df

# Establish DB connection & misc queries

In [ ]:
from main import Base, Job, PositionType, Result
from sqlalchemy import create_engine

engine = create_engine("sqlite:///database2.db")
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

Delete jobs (be careful):

## Create jobs:

In [ ]:
df_t74 = df[df["Number"].between(740000, 759999)]
min_network, max_network = df_t74.Number.min(), df_t74.Number.max()

In [ ]:
existing_jobs = set()
for job in session.query(Job).all():
    existing_jobs.add((job.position, job.network, job.setting_hash))

In [ ]:
from collections import OrderedDict
import json
settings = OrderedDict()
settings["VerboseMoveStats"] = True
settings["NodesAsPlayouts"] = True
settings["SmartPruningFactor"] = 0.0
settings["PolicyTemperature"] = 1.4
settings["FpuValue"] = 0.3
settings["FpuStrategyAtRoot"] = "absolute"
settings["FpuValueAtRoot"] = 1.0
settings["CPuct"] = 1.3
settings["CPuctAtRoot"] = 1.3
settings["MovesLeftThreshold"] = 0.0
settings["MovesLeftSlope"] = 0.007
settings["MinibatchSize"] = 1
from hashlib import blake2b
h = blake2b(digest_size=8, salt=b"a")
#h.update(repr(frozenset(settings.items())).encode())
h.update(json.dumps(settings).encode())
setting_hash = h.hexdigest()
# FIXME: The earlier hash was salted! It will produce a different one for each new python process
#setting_hash = -8915396706568843424  # T60

In [ ]:
avail_networks = set(np.unique(df.Number))
networks = set(np.arange(min_network, max_network + 1))

In [ ]:
from itertools import product

import numpy as np

#networks = list(np.around(np.linspace(min_network, max_network, num=3004)).astype(int))
#networks = list(intermediate_nets)
networks = set(np.arange(min_network, max_network + 1)) & set(np.unique(df.Number))
positions = [
    " ",
    "e2e4",
    "d2d4",
    "g1f3",
    "c2c4",
    "g2g3",
    "b2b3",
    "d2d4 d7d5",
    "e2e4 e7e6 d2d4 d7d5"
]
for n, p in product(networks, positions):
    if (p, str(n), setting_hash) not in existing_jobs:
        job = Job(
            position=p,
            network=str(n),
            position_type=PositionType.MOVES,
            setting_hash=str(setting_hash),
            settings=settings,
        )
        session.add(job)
        
fen_positions = [
    "R4n2/3p1k2/2pPp2p/1r2Pp1P/KP3Pp1/6P1/1B6/8 w - - 69 67",  # enqwert Kb3
    "3r4/1p3kp1/p1n4p/1r2p2P/4R3/4B1P1/1PR2PK1/8 w - - 0 30",  # position @ move 30
]
for n, p in product(networks, fen_positions):
    if (p, str(n), setting_hash) not in existing_jobs:
        job = Job(
            position=p,
            network=str(n),
            position_type=PositionType.FEN,
            setting_hash=str(setting_hash),
            settings=settings,
        )
        session.add(job)
    
session.commit()

# Plot results

## T74

In [ ]:
###### import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from main import Base, Job, PositionType, Result
from matplotlib import ticker
from sqlalchemy import Integer, cast, create_engine

engine = create_engine("sqlite:///database2.db")
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

position = "g1f3"
name = "g1f3"
network = "T74"
if network == "T30":
    range_lower, range_upper = 30000, 33006
elif network == "T60":
    range_lower, range_upper = 60000, 70000
elif network == "T74":
    range_lower, range_upper = 740000, 749999
results_query = session.query(Result).filter(
    Result.position == position,
    cast(Result.network, Integer) < range_upper,
    cast(Result.network, Integer) >= range_lower,
)

from scipy.special import logit

df = pd.read_sql(results_query.statement, results_query.session.bind)
df["network"] = df["network"].astype(int)
df = df.drop(columns=["position", "q_value", "setting_hash", "W", "D", "L", "M"])
# df["policy"] = logit(df["policy"])
session.close()

In [ ]:
df_pivot = df.pivot(index="network", columns="move")
df_pivot.columns = df_pivot.columns.get_level_values(1)
df_pivot = df_pivot.reindex(
    columns=df_pivot.columns[np.argsort(df_pivot.iloc[-1])[::-1]]
)
new_column = df_pivot.iloc[:, 5:].sum(axis=1)
df_pivot = df_pivot.drop(columns=df_pivot.columns[5:])
#df_pivot["others"] = new_column

In [ ]:
minnet, maxnet = df_pivot.index.min(), df_pivot.index.max()

In [ ]:
### import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns


col = plt.cm.get_cmap("Set1").colors
col = col[:5] + ((*col[7], 0.65),)
linestyles = ["-"] * 5 + ["--"]

figsize = (14.7 / 2.54, 8 / 2.54)
figsize = (5.78851 - 7.3079/72.27, 6/2.54)
figsize = (8,4)
plt.rc('text', usetex=True)
plt.rc("font", family="Latin Modern Roman")
plt.rc("xtick", labelsize=8)
plt.rc("ytick", labelsize=8)
plt.style.use(['science'])
fig, ax = plt.subplots(figsize=figsize, sharey=True)


# plt.legend(labels=df_pivot.columns, bbox_to_anchor=(1.0, 0.5, 0.5, 0.1), loc="center right", ncol=2,
#          borderaxespad=0., mode="expand")

df_pivot.plot(
    kind="line", stacked=False, ax=ax, legend=False, color=col, style=linestyles,
)

# plt.legend(labels=df_pivot.columns, bbox_to_anchor=(1.0, 0.5, 0.5, 0.1), loc="center right", ncol=2,
#          borderaxespad=0., mode="expand")
ax.set_xlabel("")
ax.set_ylabel("Policy [$\%$]")
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.01))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))
ax.xaxis.set_major_locator(ticker.MultipleLocator(200))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(50))
# ax.xaxis.set_minor_locator(ticker.MultipleLocator(500))
# ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))
ax.set_xlim(minnet, maxnet)
#ax.set_xticks([30000, 31000, 32000])


#sns.despine(fig, trim=True)
ax.grid(axis="y", ls="--", alpha=0.5)
pol_line = df_pivot.values.max()*1.05
ax.set_ylim(0, pol_line*1.06)
fsize = 8
alpha = 0.5
ax.axvline(740148, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740148-1, pol_line, "LR$\downarrow$", ha="right", size=fsize)
ax.axvline(740349, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740349-1, pol_line, "Q-threshold$\downarrow$", ha="right", size=fsize)
ax.axvline(740441, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740441-1, pol_line*0.96, "temp 0.9", ha="right", size=fsize)
ax.axvline(740602, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740602-1, pol_line, "temp 0.8", ha="right", size=fsize)
ax.axvline(740714, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740714-1, pol_line*0.96, "temp 0.7", ha="right", size=fsize)
ax.axvline(740800, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740800-1, pol_line, "decay/thr", ha="right", size=fsize)
ax.axvline(740872, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740872-1, pol_line, "LR$\downarrow$", ha="right", size=fsize)
ax.axvline(740935, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740935-1, pol_line*0.96, "MGN$\\uparrow$", ha="right", size=fsize)
ax.axvline(740981, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740981-1, pol_line, "MGN$\\uparrow$", ha="right", size=fsize)
ax.axvline(741138, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(741138+20, pol_line*0.96, "temp-endgame$\\downarrow$", ha="right", size=fsize)
ax.axvline(741255, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(741255-10, pol_line, "DBT$\\downarrow$", ha="right", size=fsize)
ax.axvline(741509, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(741509-10, pol_line*0.96, "DBT$\\uparrow$ VLW$\\uparrow$", ha="right", size=fsize)
ax.axvline(741601, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(741601-10, pol_line, "VLW $2.0$", ha="right", size=fsize)
#ax.text(741138+20, pol_line*0.96, "temp-endgame$\\downarrow$", ha="right", size=fsize)
ax.set_title(f"T74 policies after {name}")




#plt.figlegend( lines, labels, loc = 'lower center', ncol=5, labelspacing=0. )
plt.legend(labels=df_pivot.columns, ncol=5, loc="upper center", bbox_to_anchor=(0.5, -0.03),
             frameon=False)
plt.subplots_adjust(hspace=0, wspace=0.05)

plt.savefig(f"T74plot_{name}.png", dpi=200)

#plt.savefig("T74.pdf")

### Plot Q-values

In [ ]:
import numpy as np
import pandas as pd
from main import Base, Job, PositionType, Result
from scipy.special import logit
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import matplotlib.pyplot as plt
import seaborn as sns

engine = create_engine("sqlite:///database2.db")
Base.metadata.create_all(engine)


Session = sessionmaker(bind=engine)
session = Session()

position = " "
name = "start"
network = "T74"
if network == "T30":
    range_lower, range_upper = 30000, 33006
elif network == "T60":
    range_lower, range_upper = 60000, 70000
elif network == "T74":
    range_lower, range_upper = 740000, 749999
results_query = session.query(Result).filter(
    Result.position == position,
    cast(Result.network, Integer) < range_upper,
    cast(Result.network, Integer) >= range_lower,
)


df = pd.read_sql(results_query.statement, results_query.session.bind)
df = df.drop(columns=["position", "policy", "setting_hash"])
df["network"] = df["network"].astype(int)
# df["policy"] = logit(df["policy"])
session.close()

In [ ]:
df_pivot = df.pivot(index="network", columns="move")
df_pivot = df_pivot.rename(columns=dict([(x, x[1]) for x in df.columns]))

In [ ]:
minnet, maxnet = df_pivot.index.min(), df_pivot.index.max()

In [ ]:
import matplotlib.ticker as ticker

figsize = (6, 4.5)
plt.rc('text', usetex=False)
plt.rc("font", family="serif")
plt.rc("xtick", labelsize="x-small")
plt.rc("ytick", labelsize="x-small")
fig, ax = plt.subplots(figsize=figsize)

q_pivot = df.pivot(index="network", columns="move")
q_pivot.columns = q_pivot.columns.get_level_values(1)
q_pivot = q_pivot.reindex(
    columns=q_pivot.columns[np.argsort(q_pivot.iloc[-1])[::-1]]
)
q_pivot.plot(
    kind="line", stacked=False, ax=ax, colormap="tab20", legend=False, lw=0.7
)
# plt.legend(labels=[x[1] for x in df_pivot.columns], bbox_to_anchor=(0.0, -0.9, 1.0, 0.1), loc="lower center", ncol=4,
#          borderaxespad=0., mode="expand")
ax.set_xlabel("Network generation")
#ax.set_ylabel("Proportion")
#ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
#ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
#ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
ax.set_xlim(minnet, maxnet)
#ax.set_ylim(-0.2, 0.1)
ax.set_ylabel("Q value")
ax.set_title(f"Evolution of {network} Q values after {name}")
#ax.set_title(f"Evolution of {network} Q values in 7b/3k4/p5p1/Pp3p1p/1Pp2P1P/2P1R1P1/2K5/8 b - - 22 56")

ax.axvline(740148, ls="--", color="gray", zorder=-5)
ax.text(740148-1, -0.005, "LR$\downarrow$", ha="right")
ax.axvline(740349, ls="--", color="gray", zorder=-5)
ax.text(740349-1, -0.02, "Q-threshold$\downarrow$", ha="right")
plt.legend(bbox_to_anchor=(0.5, -0.1), loc="upper center", ncol=7, fontsize="smaller", frameon=False)
#sns.despine(fig, trim=True)
plt.tight_layout()
plt.savefig(f"{network}_Qvalues_{name}.png", dpi=150)

### Plot WDL

In [ ]:
###### import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from main import Base, Job, PositionType, Result
from matplotlib import ticker
from sqlalchemy import Integer, cast, create_engine

engine = create_engine("sqlite:///database2.db")
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

position = " "
name = "start"
network = "T74"
if network == "T30":
    range_lower, range_upper = 30000, 33006
elif network == "T60":
    range_lower, range_upper = 60000, 70000
elif network == "T74":
    range_lower, range_upper = 740000, 749999
results_query = session.query(Result).filter(
    Result.position == position,
    cast(Result.network, Integer) < range_upper,
    cast(Result.network, Integer) >= range_lower,
)

from scipy.special import logit

df = pd.read_sql(results_query.statement, results_query.session.bind)
df["network"] = df["network"].astype(int)
df = df.drop(columns=["position", "q_value", "setting_hash", "M"])
# df["policy"] = logit(df["policy"])
session.close()

In [ ]:
df_last = df[df["network"] == df["network"].max()]
highest_policy_move = df_last[df_last["policy"] == df_last["policy"].max()]["move"].values[0]

In [ ]:
df_filtered = df[df["move"] == highest_policy_move]
#df_filtered = df[df.groupby("network", sort=False)["policy"].transform(max) == df["policy"]]
df_filtered = df_filtered.drop(columns=["policy", "move"])

In [ ]:
import seaborn as sns
import matplotlib as mpl
fig, ax = plt.subplots(figsize=(7,4.5))
colors = sns.color_palette("Pastel1", n_colors=3)[::-1]
cmap = mpl.colors.ListedColormap(colors)
p = df_filtered.plot(
    x="network",
    kind="area",
    stacked=True,
    ax=ax,
    legend=True,
    colormap=cmap,
    linewidth=0,
    rasterized=False,
)
ax.set_xlim(df_filtered["network"].min(), df_filtered["network"].max())
ax.set_ylim(0, 1)
ax.set_xlabel("")
ax.set_ylabel("Proportion [$\%$]")
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
ax.axvline(740148, ls="--", color="gray", zorder=1)
ax.text(740148-1, 0.95, "LR$\downarrow$", ha="right")
ax.axvline(740349, ls="--", color="gray", zorder=1)
ax.text(740349-1, 0.95, "Q-threshold$\downarrow$", ha="right")
ax.axvline(740441, ls="--", color="gray", zorder=1)
ax.text(740441-1, 0.95, "temp 0.9", ha="right")
ax.axvline(740602, ls="--", color="gray", zorder=1)
ax.text(740602-1, 0.95, "temp 0.8", ha="right")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.03), ncol=3,
             frameon=False)
ax.set_title(f"WDL distribution of highest policy move after {name}")
plt.savefig(f"T74wdl_{name}.png", dpi=150)

Plot only D:

In [ ]:
import seaborn as sns
import matplotlib as mpl
df_D = df_filtered.drop(columns=["W", "L"])

fig, ax = plt.subplots(figsize=(7,4.5))
colors = sns.color_palette("Pastel1", n_colors=3)[::-1]
cmap = mpl.colors.ListedColormap(colors)
p = df_D.plot(
    x="network",
    kind="line",
    ax=ax,
    legend=False,
)
ax.set_xlim(df_filtered["network"].min(), df_filtered["network"].max())
#ax.set_ylim(bottom=0.29)
ax.set_ylim(bottom=0.1, top=0.47)
ax.set_xlabel("")
ax.set_ylabel("Proportion [$\%$]")
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
pol_line = 0.45
ax.axvline(740148, ls="--", color="gray", zorder=1)
ax.text(740148-1, pol_line, "LR$\downarrow$", ha="right")
ax.axvline(740349, ls="--", color="gray", zorder=1)
ax.text(740349-1, pol_line, "Q-threshold$\downarrow$", ha="right")
ax.axvline(740441, ls="--", color="gray", zorder=1)
ax.text(740441-1, pol_line, "temp 0.9", ha="right")
ax.axvline(740602, ls="--", color="gray", zorder=1)
ax.text(740602-1, pol_line, "temp 0.8", ha="right")
ax.axvline(740714, ls="--", color="gray", zorder=1)
ax.text(740714-1, pol_line, "temp 0.7", ha="right")
ax.axvline(740800, ls="--", color="gray", zorder=-5)
ax.text(740800-1, pol_line, "decay/thr", ha="right")
ax.axvline(740872, ls="--", color="gray", zorder=-5)
ax.text(740872-1, pol_line, "LR$\downarrow$", ha="right", size=fsize)
ax.axvline(740935, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740935-1, pol_line*0.96, "MGN$\\uparrow$", ha="right", size=fsize)
ax.axvline(740981, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740981-1, pol_line, "MGN$\\uparrow$", ha="right", size=fsize)
ax.axvline(741138, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(741138+20, pol_line*0.96, "temp-endgame$\\downarrow$", ha="right", size=fsize)
#plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.03), ncol=3,
#             frameon=False)
ax.set_title(f"Draw proportion of highest policy move after {name}")
plt.savefig(f"T74d_{name}.png", dpi=150)

## T75

In [ ]:
###### import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from main import Base, Job, PositionType, Result
from matplotlib import ticker
from sqlalchemy import Integer, cast, create_engine

engine = create_engine("sqlite:///database2.db")
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

position = " "
name = "start"
network = "T75"
if network == "T30":
    range_lower, range_upper = 30000, 33006
elif network == "T60":
    range_lower, range_upper = 60000, 70000
elif network == "T74":
    range_lower, range_upper = 740000, 749999
elif network == "T75":
    range_lower, range_upper = 750000, 759999
results_query = session.query(Result).filter(
    Result.position == position,
    cast(Result.network, Integer) < range_upper,
    cast(Result.network, Integer) >= range_lower,
)

from scipy.special import logit

df = pd.read_sql(results_query.statement, results_query.session.bind)
df["network"] = df["network"].astype(int)
df = df.drop(columns=["position", "q_value", "setting_hash", "W", "D", "L", "M"])
# df["policy"] = logit(df["policy"])
session.close()

In [ ]:
df_pivot = df.pivot(index="network", columns="move")
df_pivot.columns = df_pivot.columns.get_level_values(1)
df_pivot = df_pivot.reindex(
    columns=df_pivot.columns[np.argsort(df_pivot.iloc[-1])[::-1]]
)
new_column = df_pivot.iloc[:, 5:].sum(axis=1)
df_pivot = df_pivot.drop(columns=df_pivot.columns[5:])
#df_pivot["others"] = new_column

In [ ]:
minnet, maxnet = df_pivot.index.min(), df_pivot.index.max()

In [ ]:
### import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns


col = plt.cm.get_cmap("Set1").colors
col = col[:5] + ((*col[7], 0.65),)
linestyles = ["-"] * 5 + ["--"]

figsize = (14.7 / 2.54, 8 / 2.54)
figsize = (5.78851 - 7.3079/72.27, 6/2.54)
figsize = (8,4)
plt.rc('text', usetex=True)
plt.rc("font", family="Latin Modern Roman")
plt.rc("xtick", labelsize=8)
plt.rc("ytick", labelsize=8)
plt.style.use(['science'])
fig, ax = plt.subplots(figsize=figsize, sharey=True)


# plt.legend(labels=df_pivot.columns, bbox_to_anchor=(1.0, 0.5, 0.5, 0.1), loc="center right", ncol=2,
#          borderaxespad=0., mode="expand")

df_pivot.plot(
    kind="line", stacked=False, ax=ax, legend=False, color=col, style=linestyles,
)

# plt.legend(labels=df_pivot.columns, bbox_to_anchor=(1.0, 0.5, 0.5, 0.1), loc="center right", ncol=2,
#          borderaxespad=0., mode="expand")
ax.set_xlabel("")
ax.set_ylabel("Policy [$\%$]")
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.01))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
# ax.xaxis.set_minor_locator(ticker.MultipleLocator(500))
# ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))
ax.set_xlim(minnet, maxnet)
#ax.set_xticks([30000, 31000, 32000])


#sns.despine(fig, trim=True)
ax.grid(axis="y", ls="--", alpha=0.5)
pol_line = df_pivot.values.max()*1.05
ax.set_ylim(0, pol_line*1.06)
fsize = 8
alpha = 0.5
ax.axvline(750069, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(750069-1, pol_line, "DBT $0.5$", ha="right", size=fsize)
ax.axvline(750089, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(750089-1, pol_line, "DBT $0.25$", ha="right", size=fsize)
ax.axvline(750123, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(750123-1, pol_line, "DBT $0.15$", ha="right", size=fsize)

ax.text(750173-1, pol_line, "DBT $0.1$", ha="right", size=fsize)
ax.axvline(750173, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(750194-1, pol_line, "\\noindent VLW $1.0$\\\\MGN $0.85$", ha="right", size=fsize)
ax.axvline(750194, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(750213-1, pol_line, "\\noindent LR $0.04$\\\\MGN $1.7$", ha="right", size=fsize)
ax.axvline(750213, ls="--", color="gray", zorder=-5, alpha=alpha)

#ax.text(741138+20, pol_line*0.96, "temp-endgame$\\downarrow$", ha="right", size=fsize)
ax.set_title(f"T75 policies after {name}")




#plt.figlegend( lines, labels, loc = 'lower center', ncol=5, labelspacing=0. )
plt.legend(labels=df_pivot.columns, ncol=5, loc="upper center", bbox_to_anchor=(0.5, -0.03),
             frameon=False)
plt.subplots_adjust(hspace=0, wspace=0.05)

plt.savefig(f"T75plot_{name}.png", dpi=200)

#plt.savefig("T74.pdf")

### Plot Q-values

In [ ]:
import numpy as np
import pandas as pd
from main import Base, Job, PositionType, Result
from scipy.special import logit
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import matplotlib.pyplot as plt
import seaborn as sns

engine = create_engine("sqlite:///database2.db")
Base.metadata.create_all(engine)


Session = sessionmaker(bind=engine)
session = Session()

position = " "
name = "start"
network = "T74"
if network == "T30":
    range_lower, range_upper = 30000, 33006
elif network == "T60":
    range_lower, range_upper = 60000, 70000
elif network == "T74":
    range_lower, range_upper = 740000, 749999
results_query = session.query(Result).filter(
    Result.position == position,
    cast(Result.network, Integer) < range_upper,
    cast(Result.network, Integer) >= range_lower,
)


df = pd.read_sql(results_query.statement, results_query.session.bind)
df = df.drop(columns=["position", "policy", "setting_hash"])
df["network"] = df["network"].astype(int)
# df["policy"] = logit(df["policy"])
session.close()

In [ ]:
df_pivot = df.pivot(index="network", columns="move")
df_pivot = df_pivot.rename(columns=dict([(x, x[1]) for x in df.columns]))

In [ ]:
minnet, maxnet = df_pivot.index.min(), df_pivot.index.max()

In [ ]:
import matplotlib.ticker as ticker

figsize = (6, 4.5)
plt.rc('text', usetex=False)
plt.rc("font", family="serif")
plt.rc("xtick", labelsize="x-small")
plt.rc("ytick", labelsize="x-small")
fig, ax = plt.subplots(figsize=figsize)

q_pivot = df.pivot(index="network", columns="move")
q_pivot.columns = q_pivot.columns.get_level_values(1)
q_pivot = q_pivot.reindex(
    columns=q_pivot.columns[np.argsort(q_pivot.iloc[-1])[::-1]]
)
q_pivot.plot(
    kind="line", stacked=False, ax=ax, colormap="tab20", legend=False, lw=0.7
)
# plt.legend(labels=[x[1] for x in df_pivot.columns], bbox_to_anchor=(0.0, -0.9, 1.0, 0.1), loc="lower center", ncol=4,
#          borderaxespad=0., mode="expand")
ax.set_xlabel("Network generation")
#ax.set_ylabel("Proportion")
#ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
#ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
#ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
ax.set_xlim(minnet, maxnet)
#ax.set_ylim(-0.2, 0.1)
ax.set_ylabel("Q value")
ax.set_title(f"Evolution of {network} Q values after {name}")
#ax.set_title(f"Evolution of {network} Q values in 7b/3k4/p5p1/Pp3p1p/1Pp2P1P/2P1R1P1/2K5/8 b - - 22 56")

ax.axvline(740148, ls="--", color="gray", zorder=-5)
ax.text(740148-1, -0.005, "LR$\downarrow$", ha="right")
ax.axvline(740349, ls="--", color="gray", zorder=-5)
ax.text(740349-1, -0.02, "Q-threshold$\downarrow$", ha="right")
plt.legend(bbox_to_anchor=(0.5, -0.1), loc="upper center", ncol=7, fontsize="smaller", frameon=False)
#sns.despine(fig, trim=True)
plt.tight_layout()
plt.savefig(f"{network}_Qvalues_{name}.png", dpi=150)

### Plot WDL

In [ ]:
###### import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from main import Base, Job, PositionType, Result
from matplotlib import ticker
from sqlalchemy import Integer, cast, create_engine

engine = create_engine("sqlite:///database2.db")
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

position = " "
name = "start"
network = "T74"
if network == "T30":
    range_lower, range_upper = 30000, 33006
elif network == "T60":
    range_lower, range_upper = 60000, 70000
elif network == "T74":
    range_lower, range_upper = 740000, 749999
results_query = session.query(Result).filter(
    Result.position == position,
    cast(Result.network, Integer) < range_upper,
    cast(Result.network, Integer) >= range_lower,
)

from scipy.special import logit

df = pd.read_sql(results_query.statement, results_query.session.bind)
df["network"] = df["network"].astype(int)
df = df.drop(columns=["position", "q_value", "setting_hash", "M"])
# df["policy"] = logit(df["policy"])
session.close()

In [ ]:
df_last = df[df["network"] == df["network"].max()]
highest_policy_move = df_last[df_last["policy"] == df_last["policy"].max()]["move"].values[0]

In [ ]:
df_filtered = df[df["move"] == highest_policy_move]
#df_filtered = df[df.groupby("network", sort=False)["policy"].transform(max) == df["policy"]]
df_filtered = df_filtered.drop(columns=["policy", "move"])

In [ ]:
import seaborn as sns
import matplotlib as mpl
fig, ax = plt.subplots(figsize=(7,4.5))
colors = sns.color_palette("Pastel1", n_colors=3)[::-1]
cmap = mpl.colors.ListedColormap(colors)
p = df_filtered.plot(
    x="network",
    kind="area",
    stacked=True,
    ax=ax,
    legend=True,
    colormap=cmap,
    linewidth=0,
    rasterized=False,
)
ax.set_xlim(df_filtered["network"].min(), df_filtered["network"].max())
ax.set_ylim(0, 1)
ax.set_xlabel("")
ax.set_ylabel("Proportion [$\%$]")
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
ax.axvline(740148, ls="--", color="gray", zorder=1)
ax.text(740148-1, 0.95, "LR$\downarrow$", ha="right")
ax.axvline(740349, ls="--", color="gray", zorder=1)
ax.text(740349-1, 0.95, "Q-threshold$\downarrow$", ha="right")
ax.axvline(740441, ls="--", color="gray", zorder=1)
ax.text(740441-1, 0.95, "temp 0.9", ha="right")
ax.axvline(740602, ls="--", color="gray", zorder=1)
ax.text(740602-1, 0.95, "temp 0.8", ha="right")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.03), ncol=3,
             frameon=False)
ax.set_title(f"WDL distribution of highest policy move after {name}")
plt.savefig(f"T74wdl_{name}.png", dpi=150)

Plot only D:

In [ ]:
import seaborn as sns
import matplotlib as mpl
df_D = df_filtered.drop(columns=["W", "L"])

fig, ax = plt.subplots(figsize=(7,4.5))
colors = sns.color_palette("Pastel1", n_colors=3)[::-1]
cmap = mpl.colors.ListedColormap(colors)
p = df_D.plot(
    x="network",
    kind="line",
    ax=ax,
    legend=False,
)
ax.set_xlim(df_filtered["network"].min(), df_filtered["network"].max())
#ax.set_ylim(bottom=0.29)
ax.set_ylim(bottom=0.1, top=0.47)
ax.set_xlabel("")
ax.set_ylabel("Proportion [$\%$]")
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
pol_line = 0.45
ax.axvline(740148, ls="--", color="gray", zorder=1)
ax.text(740148-1, pol_line, "LR$\downarrow$", ha="right")
ax.axvline(740349, ls="--", color="gray", zorder=1)
ax.text(740349-1, pol_line, "Q-threshold$\downarrow$", ha="right")
ax.axvline(740441, ls="--", color="gray", zorder=1)
ax.text(740441-1, pol_line, "temp 0.9", ha="right")
ax.axvline(740602, ls="--", color="gray", zorder=1)
ax.text(740602-1, pol_line, "temp 0.8", ha="right")
ax.axvline(740714, ls="--", color="gray", zorder=1)
ax.text(740714-1, pol_line, "temp 0.7", ha="right")
ax.axvline(740800, ls="--", color="gray", zorder=-5)
ax.text(740800-1, pol_line, "decay/thr", ha="right")
ax.axvline(740872, ls="--", color="gray", zorder=-5)
ax.text(740872-1, pol_line, "LR$\downarrow$", ha="right", size=fsize)
ax.axvline(740935, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740935-1, pol_line*0.96, "MGN$\\uparrow$", ha="right", size=fsize)
ax.axvline(740981, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(740981-1, pol_line, "MGN$\\uparrow$", ha="right", size=fsize)
ax.axvline(741138, ls="--", color="gray", zorder=-5, alpha=alpha)
ax.text(741138+20, pol_line*0.96, "temp-endgame$\\downarrow$", ha="right", size=fsize)
#plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.03), ncol=3,
#             frameon=False)
ax.set_title(f"Draw proportion of highest policy move after {name}")
plt.savefig(f"T74d_{name}.png", dpi=150)